# Sample chatbot response

We first start with sample chat bot response without any optimization of using any special technique.

In [37]:
from typing import List, Dict, Tuple, Optional, Any
import json

In [38]:
from ollama import chat
from ollama import ChatResponse
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
)

MODEL_NAME: str = "llama3.1"  # tinyllama


class LLM:
    def __init__(self, model_name: str) -> None:
        self.model_name = model_name

    def chat(self, messages: List[Dict[str, str]]) -> str:
        response: ChatResponse = chat(
            model=self.model_name,
            messages=messages,
        )
        return response.message.content

    def chat_with_user(
        self,
        user_message: str,
        context: Optional[List[str]] = None,
        Previous_messages: Optional[List[str]] = None,
    ) -> str:
        messages = [
            {
                "role": "system",
                "content": "You are a chatbot. Your job is to answer to the users query as per the instructions.",
            }
        ]

        if Previous_messages:
            messages.append(
                {
                    "role": "system",
                    "content": f"IMPORTANT: You MUST answer strictly adher to the following instruction for answering {' '.join(Previous_messages).upper()} \n\n",
                }
            )
        if context:
            context_text = "\n".join(context)
            messages[0]["content"] += (
                f"\n\n You have additional information to improve your answer:\n{context_text}"
            )

        messages.append(
            {"role": "user", "content": user_message},
        )

        print(
            f"Overall prompt: =================================================================>\n {messages}"
        )
        response: ChatResponse = chat(model=self.model_name, messages=messages)
        return response.message.content


# Sample usage.

llm = LLM(model_name=MODEL_NAME)
print(
    llm.chat_with_user(
        "My dog died few days ago. I am feeling void. What should I do to fill my void.!",
        None,
        ["Make sure you answer in portugese language."],
    )
)

Overall prompt: =================================================================>
 [{'role': 'system', 'content': 'You are a chatbot. Your job is to answer to the users query as per the instructions.'}, {'role': 'system', 'content': 'IMPORTANT: You MUST answer strictly adher to the following instruction for answering MAKE SURE YOU ANSWER IN PORTUGESE LANGUAGE. \n\n'}, {'role': 'user', 'content': 'My dog died few days ago. I am feeling void. What should I do to fill my void.!'}]


2025-03-26 18:01:01,073 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Lamento ouvir que o seu cão tenha falecido recentemente. O sentimento de vazio é comum após a perda de um ente querido.

Existem várias coisas que você pode fazer para preencher esse vazio:

1. **Compartilhe seus sentimentos**: Sente-se confortável e compartilhe suas emoções com amigos, familiares ou um profissional de saúde mental. Isso pode ajudar a aliviar o sentimento de isolamento.
2. **Cuide da si mesmo**: Tente manter uma rotina saudável, comendo bem, dormindo regularmente e fazendo exercícios físicos. Isso pode ajudar a melhorar sua humor e energia.
3. **Faça algo criativo**: Pinte, escreva, faça música ou pratique outra atividade artística. Isso pode ser uma maneira saudável de lidar com as emoções e criar algo positivo.
4. **Passe tempo ao ar livre**: Faça um passeio, entre em contato com a natureza e permita que o exercício ajudem a melhorar seu humor.
5. **Pense nos bons momentos**: Reflita sobre os bons momentos que você compartilhou com o seu cão. Isso pode ser uma maneir

# DEMO rank ranking algorithms for documents.

A Demo Rank Algorithm is a simplified ranking mechanism used to order items based on predefined criteria. It typically assigns scores to items based on relevance, importance, or similarity to a given query. In information retrieval, ranking algorithms like BM25, PageRank, or Cosine Similarity are commonly used to sort search results.

For example, in a cosine similarity-based ranking system, documents are ranked by computing their similarity to a query vector. The higher the similarity score, the higher the document appears in the results.

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Dict


class DocumentRanker:
    def __init__(self, documents: List[Dict[str, str]]) -> None:
        """
        Initialize the DocumentRanker with a List of documents.
        Each document should be a dictionary with 'id' and 'content' keys.
        """
        self.documents: List[Dict[str, str]] = documents
        self.vectorizer: TfidfVectorizer = TfidfVectorizer(stop_words="english")
        self.doc_term_matrix = self.vectorizer.fit_transform(
            [doc["problem"] for doc in documents]
        )

    def rank_documents(
        self, query: str, limit: int = 3, threshold: float = 0.4
    ) -> List[Tuple[Dict[str, str], float]]:
        """
        Rank documents based on their relevance to the query using cosine similarity.
        :param query: The query string to compare against the documents.
        :param limit: The number of top documents to return.
        :return: A List of ranked documents with their scores.
        """
        query_vector = self.vectorizer.transform([query])
        scores = cosine_similarity(query_vector, self.doc_term_matrix).flatten()

        ranked_documents = sorted(
            zip(self.documents, scores), key=lambda x: x[1], reverse=True
        )

        # Dynamic threshold adjustment
        max_score = max(scores) if scores.size > 0 else 0
        adjusted_threshold = max(threshold, max_score * 0.5)

        ranked_documents = [
            (doc, score)
            for doc, score in ranked_documents
            if score > adjusted_threshold
        ]
        return ranked_documents[:limit]

    def display_ranked_documents(
        self, ranked_documents: List[Tuple[Dict[str, str], float]]
    ) -> None:
        """
        Display the ranked documents in a readable format.
        :param ranked_documents: A List of tuples containing documents and their scores.
        """
        print("Ranked Documents:")
        for rank, (doc, score) in enumerate(ranked_documents, start=1):
            print(f"Rank {rank}: Document ID {doc['id']} with score {score:.4f}")
            print(f"Content: {doc['solution']}\n")

In [40]:
# Sample demonstration of the DocumentRanker class

documents: List[Dict[str, Any]] = []

with open("files/docs.json", "r") as json_file:
    data = json.load(json_file)
    for doc in data:
        documents.append(doc)

queries = [
    "My only pet died last week. I am feeling very lonely. What should I do?",
    "My wife cheated on me, and she filled for a divorce to get alimony. I am feeling very lonely. What should I do?",
    "I do not have a good job to support my family at present.",
    "My boss harrases me. I am not recognized at my workplace.",
]

for idx, q in enumerate(queries):
    print(f"Index: {idx} , Query: {q}")
    ranker = DocumentRanker(documents)
    ranked_docs = ranker.rank_documents(q)
    ranker.display_ranked_documents(ranked_docs)
    print("=====================================================")

Index: 0 , Query: My only pet died last week. I am feeling very lonely. What should I do?
Ranked Documents:
Rank 1: Document ID 83 with score 0.6456
Content: Solution: Hello How may i help you today? I am sorry I know exactly how you are feeling I have had a dog that i loved more than anything die couple years ago What movie was it? Oh ok how did your dog die was something wrong with him or just old age? I am really sorry that has got to be the worst feeling putting your dog to sleep I just think of it this way your dog is in dog heaven and is watching over you that is how i think of it even though it really aint true thats just how i think My dog died of old age  it took me like 2 or 3 weeks to get over it and not cry for once  I can understand how you feeling but everything will be okay and it is ok to miss her  I understand I would break down to if I saw that  Just do something you like to do to get your mind off of it  Right it is always good to have someone to talk to in a situati

# Add vector semantic search 


Vector semantic search uses vector embeddings to find relevant results based on meaning rather than exact keywords.

Text is converted into high-dimensional vector representations using models like Word2Vec, BERT, or OpenAI embeddings.

A query is also transformed into a vector, and similarity (e.g., cosine similarity) is used to rank results.

## Advantages:
- Finds results based on context and meaning rather than exact matches.
- Handles synonyms and paraphrasing well.
- Supports multimodal search (text, images, audio).

## Use Cases:

- Search engines (e.g., Google, e-commerce search).
- Chatbots and AI assistants for intent matching.
- Recommendation systems (suggesting similar products, articles).

We are using Qdrant an open source vector database.

In [41]:
import logging
from collections import OrderedDict
import uuid

from qdrant_client import QdrantClient, models
from qdrant_client.models import PointStruct
from qdrant_client.http.models import VectorParams, Distance
import ollama


class SemanticEmbeddingService:
    def __init__(self, cache_size: int = 1000) -> None:
        self.embeddings_cache: OrderedDict[str, List[float]] = OrderedDict()
        self.cache_size = cache_size

    def get_embeddings(self, text: str) -> List[float]:
        if text in self.embeddings_cache:
            self.embeddings_cache.move_to_end(text)
            return self.embeddings_cache[text]
        else:
            response = ollama.embed(model="mxbai-embed-large", input=text)
            self.embeddings_cache[text] = response.embeddings[0]
            logging.info(f"Embedding for {text} is {str(response.embeddings[0])}")
            if len(self.embeddings_cache) > self.cache_size:
                self.embeddings_cache.popitem(last=False)

            return self.embeddings_cache[text]

In [42]:
# Initialize the SemanticEmbeddingService with a cache size of 1000
embedding_service = SemanticEmbeddingService(cache_size=1000)


for idx, q in enumerate(queries):
    # Get embeddings for the text
    embeddings = embedding_service.get_embeddings(q)

    # Print the embeddings
    print(f"Embeddings for '{q}': {embeddings}")

2025-03-26 18:01:02,723 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-03-26 18:01:02,724 [INFO] Embedding for My only pet died last week. I am feeling very lonely. What should I do? is [0.029631576, 0.005761803, -0.048718598, 0.011932161, -0.027410926, -0.05784085, 0.06425013, 0.029442623, 0.039420884, -0.0067058834, 0.017432792, -0.044298716, -0.03885782, 0.009484146, -0.028376171, -0.0606311, 0.0033278384, -0.030246567, -0.018542148, 0.041980848, -0.029785909, 0.059403546, -0.025356539, -0.0054445835, 0.010525575, 0.0117527805, -0.005524074, 0.007989732, 0.030148914, 0.043823294, -0.039667353, 0.01379165, -0.005483564, -0.046023592, 0.015953735, -0.003980094, -0.020871537, -0.0093919765, -0.0300329, -0.029849704, 0.034332823, -0.017756378, 0.06488117, -0.051664226, -0.009517218, 0.010527802, 0.010733082, -0.010714066, 0.039865054, -0.062029958, -0.030588754, 0.0032706927, -0.04333777, -0.011641216, 0.052408576, -0.06469134, 0.027447829, 0.036498882

Embeddings for 'My only pet died last week. I am feeling very lonely. What should I do?': [0.029631576, 0.005761803, -0.048718598, 0.011932161, -0.027410926, -0.05784085, 0.06425013, 0.029442623, 0.039420884, -0.0067058834, 0.017432792, -0.044298716, -0.03885782, 0.009484146, -0.028376171, -0.0606311, 0.0033278384, -0.030246567, -0.018542148, 0.041980848, -0.029785909, 0.059403546, -0.025356539, -0.0054445835, 0.010525575, 0.0117527805, -0.005524074, 0.007989732, 0.030148914, 0.043823294, -0.039667353, 0.01379165, -0.005483564, -0.046023592, 0.015953735, -0.003980094, -0.020871537, -0.0093919765, -0.0300329, -0.029849704, 0.034332823, -0.017756378, 0.06488117, -0.051664226, -0.009517218, 0.010527802, 0.010733082, -0.010714066, 0.039865054, -0.062029958, -0.030588754, 0.0032706927, -0.04333777, -0.011641216, 0.052408576, -0.06469134, 0.027447829, 0.036498882, 0.031795908, 0.006815877, 0.021559257, -0.0030452576, -0.0046039447, -0.05501559, 0.0030213173, 0.042240445, -0.060388595, 0.0256

2025-03-26 18:01:03,014 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-03-26 18:01:03,015 [INFO] Embedding for I do not have a good job to support my family at present. is [-0.031541437, -0.014230062, -0.0789957, 0.0061065024, 0.030266115, -0.05084275, 0.0047561587, 0.025485292, 0.06898517, 0.003596477, 0.039597422, -0.03418797, 0.034405537, -0.038382854, -0.040431365, -0.009536076, -0.003360921, -0.034983385, -0.027008217, 0.008975196, -0.002255469, 0.01609482, -0.0068212375, 0.0071400916, 0.0123617835, 0.01878659, 0.015415534, 0.001328067, 0.015561687, 0.032866277, -0.02844353, -0.06897172, -0.013260367, -0.031312454, -0.00037813958, 0.00043156266, -0.0076748617, -0.020113092, 0.0053343694, -0.03617969, 0.0050724605, -0.012603325, 0.07057325, -0.015039427, -0.0046964376, -0.041623205, -0.013191501, -0.0032053567, 0.015822409, -0.02139859, 0.010509061, 0.006141219, 0.01056653, -0.010090909, 0.07231252, -0.0034566994, 0.018893946, 0.02436999, -0.00409

Embeddings for 'I do not have a good job to support my family at present.': [-0.031541437, -0.014230062, -0.0789957, 0.0061065024, 0.030266115, -0.05084275, 0.0047561587, 0.025485292, 0.06898517, 0.003596477, 0.039597422, -0.03418797, 0.034405537, -0.038382854, -0.040431365, -0.009536076, -0.003360921, -0.034983385, -0.027008217, 0.008975196, -0.002255469, 0.01609482, -0.0068212375, 0.0071400916, 0.0123617835, 0.01878659, 0.015415534, 0.001328067, 0.015561687, 0.032866277, -0.02844353, -0.06897172, -0.013260367, -0.031312454, -0.00037813958, 0.00043156266, -0.0076748617, -0.020113092, 0.0053343694, -0.03617969, 0.0050724605, -0.012603325, 0.07057325, -0.015039427, -0.0046964376, -0.041623205, -0.013191501, -0.0032053567, 0.015822409, -0.02139859, 0.010509061, 0.006141219, 0.01056653, -0.010090909, 0.07231252, -0.0034566994, 0.018893946, 0.02436999, -0.004091607, -0.020729033, 0.01815322, 0.020920303, 0.014186036, -0.04724401, -0.0046578175, 0.017685598, 0.014012326, 0.040341005, 0.0183

In [43]:
COLLECTION_NAME = "emotional_chatbot"


class SemanticQdrantService:
    def __init__(self, url: str) -> None:
        self.client = QdrantClient(url=url)

    def collection_exists(self, collection_name) -> bool:
        try:
            response = self.client.get_collection(collection_name)
            return response is not None
        except Exception:
            return False

    def create_collection(self, collection_name: str) -> None:
        print(
            f"If the collection {collection_name} does not exist, create it. {self.collection_exists(collection_name)}"
        )
        if self.collection_exists(collection_name) is False:
            self.client.create_collection(
                collection_name=collection_name,
                vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
            )

    def delete_collection(self, collection_name: str) -> None:
        if self.collection_exists(collection_name):
            self.client.delete_collection(collection_name)

    def upsert_points(self, collection_name: str, points: List[PointStruct]) -> None:
        self.client.upsert(collection_name=collection_name, points=points)

    def search(
        self,
        query_embedding: List[float],
        collection_name: str = "emotional_chatbot",
        limit: int = 3,
    ) -> List[PointStruct]:
        return self.client.search(
            collection_name=collection_name,
            query_vector=query_embedding,
            limit=limit,
        )

In [44]:
# Initialize the service
url = "http://localhost:6333"  # Replace with your Qdrant URL
service = SemanticQdrantService(url=url)

# Create a collection if it doesn't exist
collection_name = "emotional_chatbot"
service.delete_collection(collection_name)
service.create_collection(collection_name)

2025-03-26 18:01:03,198 [INFO] HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-03-26 18:01:03,200 [INFO] HTTP Request: GET http://localhost:6333/collections/emotional_chatbot "HTTP/1.1 404 Not Found"
2025-03-26 18:01:03,201 [INFO] HTTP Request: GET http://localhost:6333/collections/emotional_chatbot "HTTP/1.1 404 Not Found"
2025-03-26 18:01:03,202 [INFO] HTTP Request: GET http://localhost:6333/collections/emotional_chatbot "HTTP/1.1 404 Not Found"


If the collection emotional_chatbot does not exist, create it. False


2025-03-26 18:01:03,366 [INFO] HTTP Request: PUT http://localhost:6333/collections/emotional_chatbot "HTTP/1.1 200 OK"


In [45]:
# Sample usage of the SemanticQdrantService class


from qdrant_client.models import PointStruct


def use_semantic_qdrant_service():
    # Initialize the embedding service
    embedding_service = SemanticEmbeddingService(cache_size=1000)

    # Example texts to generate embeddings for
    text1 = "Problem: Negative self-talk and self-criticism can make you feel unworthy of care and support."
    text2 = "Problem: Feeling disconnected from your emotions can lead to confusion and distress"

    # Get embeddings for the texts
    embedding1 = embedding_service.get_embeddings(text1)
    embedding2 = embedding_service.get_embeddings(text2)

    # Initialize the Qdrant service
    url = "http://localhost:6333"  # Replace with your Qdrant URL
    service = SemanticQdrantService(url=url)

    # Create a collection if it doesn't exist
    collection_name = "emotional_chatbot"
    service.create_collection(collection_name)

    # Upsert points into the collection
    points = [
        PointStruct(
            id=1,
            vector=embedding1,
            payload={
                "solution": "Solution: Treat yourself as you would treat a friend. Practice self-compassion by using kind words and forgiving yourself for mistakes.",
                "problem": "Problem: Negative self-talk and self-criticism can make you feel unworthy of care and support.",
            },
        ),
        PointStruct(
            id=2,
            vector=embedding2,
            payload={
                "solution": "Solution: Practice mindfulness to observe your thoughts and feelings without judgment. This can help you understand your emotions better.",
                "problem": "Problem: Feeling disconnected from your emotions can lead to confusion and distress.",
            },
        ),
    ]
    service.upsert_points(collection_name=collection_name, points=points)

    # Perform a search
    query_text = "How are you feeling today?"
    query_embedding = embedding_service.get_embeddings(query_text)
    results = service.search(query_embedding=query_embedding, limit=5)

    # Print the search results
    for result in results:
        print(f"ID: {result.id}, Score: {result.score}, Payload: {result.payload}")


# Call the function
use_semantic_qdrant_service()

2025-03-26 18:01:03,500 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-03-26 18:01:03,501 [INFO] Embedding for Problem: Negative self-talk and self-criticism can make you feel unworthy of care and support. is [-0.0034734902, -0.023247039, -0.0024354414, 0.037755344, 0.008179359, -0.08410617, 0.02124062, 0.0171219, 0.0017203758, 0.017487608, 0.013396573, -0.023295943, 0.029548561, -0.034505855, -0.015959801, -0.022106439, -0.032786634, -0.011965247, -0.07148796, 0.015202752, 0.011591141, 0.009972728, 0.021256384, 0.044870567, -0.003629505, 0.025219431, -0.026684957, 0.01722426, 0.04312586, 0.019151539, -0.015541271, -0.004852122, -0.017688284, -0.04358938, 0.033870053, -0.08610427, 0.004447754, -0.04515212, -0.0031237297, -0.012027016, 0.0041865357, -0.004058757, 0.03284673, -0.037307434, -0.035572805, -0.025604082, -0.0313978, -0.012558887, 0.0039707376, -0.026324637, -0.0058135493, -0.00019170954, 0.023428276, 0.025733717, 0.034833316, -0.019963203, 

If the collection emotional_chatbot does not exist, create it. True
ID: 2, Score: 0.5781844, Payload: {'solution': 'Solution: Practice mindfulness to observe your thoughts and feelings without judgment. This can help you understand your emotions better.', 'problem': 'Problem: Feeling disconnected from your emotions can lead to confusion and distress.'}
ID: 1, Score: 0.536466, Payload: {'solution': 'Solution: Treat yourself as you would treat a friend. Practice self-compassion by using kind words and forgiving yourself for mistakes.', 'problem': 'Problem: Negative self-talk and self-criticism can make you feel unworthy of care and support.'}


In [46]:
class SemanticSearchRepo:
    def __init__(
        self,
        embedding_service: SemanticEmbeddingService,
        qdrant_service: SemanticQdrantService,
    ):
        self.embedding_service = embedding_service
        self.qdrant_service = qdrant_service

    def prepare_points(
        self, texts: List[str], metadata: List[Dict]
    ) -> List[PointStruct]:
        return [
            PointStruct(
                id=str(uuid.uuid4()),
                vector=self.embedding_service.get_embeddings(text),
                payload={"text": text, **meta},
            )
            for _, (text, meta) in enumerate(zip(texts, metadata))
        ]

    async def create_collection(self, collection_name: str):
        collection_exists = self.qdrant_service.client.collection_exists(
            collection_name=collection_name
        )
        if collection_exists is False:
            self.qdrant_service.client.create_collection(
                collection_name,
                vectors_config=models.VectorParams(
                    size=1024, distance=models.Distance.COSINE
                ),
            )

    def initialize_qdrant(
        self,
        texts: List[str],
        metadata: List[Dict[str, str]],
        collection_name: str = "emotional_chatbot",
    ) -> bool:
        points = self.prepare_points(texts, metadata)
        self.qdrant_service.upsert_points(collection_name, points)
        return True

    def query_text(
        self,
        query_text: str,
        collection_name: str = "emotional_chatbot",
        threshold: float = 0.6,
    ) -> List[Dict]:
        query_embedding = self.embedding_service.get_embeddings(query_text)
        response = self.qdrant_service.search(query_embedding, collection_name)
        logging.info(f"Query: {query_text}")
        result = []

        logging.info(f"Response: {response}")
        for data in response:
            if data.score > threshold:
                entry = {
                    "score": data.score,
                    "content": data.payload["content"],
                    "metadata": data.payload,
                }
                if entry not in result:
                    result.append(entry)

        print(f"The result is {result}")
        return result

In [47]:
with open("files/docs.json", "r") as json_file:
    data = json.load(json_file)
    for doc in data:
        documents.append(doc)

    # Initialize the required services
embedding_service = SemanticEmbeddingService()
qdrant_service = SemanticQdrantService(url=url)

# Create an instance of SemanticSearchRepo
semantic_repo = SemanticSearchRepo(embedding_service, qdrant_service)


def sample_usage():
    # Prepare the data
    metadata: List[Dict[str, str]] = [
        {"content": item["solution"], "problem": item["problem"]} for item in documents
    ]
    texts: List[str] = [item["problem"] for item in metadata]
    # # Initialize Qdrant with the sample data
    semantic_repo.initialize_qdrant(texts, metadata)


sample_usage()

2025-03-26 18:01:03,844 [INFO] HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-03-26 18:01:03,948 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-03-26 18:01:03,949 [INFO] Embedding for Problem: I hate my job but I am scared to quit and seek a new career. is [0.005505477, -0.036455214, -0.040857375, 0.022028117, 0.022901272, -0.08164696, 0.018757485, 0.018596955, 0.016051834, 0.051040575, 0.041258, -0.0030479014, 0.017448466, -0.050878238, 0.011304908, 0.024777126, -0.00448847, 0.019010475, -0.077044636, 0.007614368, 0.0017883914, 0.009632897, 0.012259425, -0.021917198, -0.024969276, 0.0113849975, -0.019290946, -0.013810826, -0.015083937, 0.039519988, -0.024050102, -0.033936586, -0.035439275, -0.03762695, -0.00048261313, -0.0476458, 0.022651408, -0.017981453, -0.024635673, -0.018320756, -0.024077844, 0.0066998163, 0.040640295, -0.047809828, -0.00830825, -0.046201676, -0.042258527, -0.008908035, 0.013313237, -0.032523893, 0.02008098, 0.04

In [48]:
for idx, q in enumerate(queries):
    # Query the data

    threshold = 0.7
    results = semantic_repo.query_text(q, threshold=threshold)

    # Print the results
    print("Query Results:")
    for idx, result in enumerate(results):
        print(f"{idx + 1}. Score: {result['score']:.4f}")

2025-03-26 18:01:40,218 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-03-26 18:01:40,219 [INFO] Embedding for My only pet died last week. I am feeling very lonely. What should I do? is [0.029631576, 0.005761803, -0.048718598, 0.011932161, -0.027410926, -0.05784085, 0.06425013, 0.029442623, 0.039420884, -0.0067058834, 0.017432792, -0.044298716, -0.03885782, 0.009484146, -0.028376171, -0.0606311, 0.0033278384, -0.030246567, -0.018542148, 0.041980848, -0.029785909, 0.059403546, -0.025356539, -0.0054445835, 0.010525575, 0.0117527805, -0.005524074, 0.007989732, 0.030148914, 0.043823294, -0.039667353, 0.01379165, -0.005483564, -0.046023592, 0.015953735, -0.003980094, -0.020871537, -0.0093919765, -0.0300329, -0.029849704, 0.034332823, -0.017756378, 0.06488117, -0.051664226, -0.009517218, 0.010527802, 0.010733082, -0.010714066, 0.039865054, -0.062029958, -0.030588754, 0.0032706927, -0.04333777, -0.011641216, 0.052408576, -0.06469134, 0.027447829, 0.036498882

The result is [{'score': 0.83924747, 'content': 'Solution: That is a very hard thing to be going through where you very close to your cat?\n I am very sorry to hear that i have also lost a pet earlier this year his name was fish and was only 3 years old he unfortunately passed due to liver cancer. It was a very trying experience but slowly the pain of his passing has got a little bit easier.\n Have you tried talking about this loss to friends of family? What have your feelings been like after the passing of your cat? Those are completely valid feelings to have at this time Have you considered getting a small reminder of your cat perhaps a small shrine or tribute to him/her? Her I apologize for the confusion. Expressing your emotion is very healthy for example maybe taking up a new hobby or trying something new that scares or excites you to get your mind off of this difficult time. What was your favorite genre of video games?\n Have you considered playing again possibly joining an onlin

2025-03-26 18:01:40,518 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-03-26 18:01:40,519 [INFO] Embedding for I do not have a good job to support my family at present. is [-0.031541437, -0.014230062, -0.0789957, 0.0061065024, 0.030266115, -0.05084275, 0.0047561587, 0.025485292, 0.06898517, 0.003596477, 0.039597422, -0.03418797, 0.034405537, -0.038382854, -0.040431365, -0.009536076, -0.003360921, -0.034983385, -0.027008217, 0.008975196, -0.002255469, 0.01609482, -0.0068212375, 0.0071400916, 0.0123617835, 0.01878659, 0.015415534, 0.001328067, 0.015561687, 0.032866277, -0.02844353, -0.06897172, -0.013260367, -0.031312454, -0.00037813958, 0.00043156266, -0.0076748617, -0.020113092, 0.0053343694, -0.03617969, 0.0050724605, -0.012603325, 0.07057325, -0.015039427, -0.0046964376, -0.041623205, -0.013191501, -0.0032053567, 0.015822409, -0.02139859, 0.010509061, 0.006141219, 0.01056653, -0.010090909, 0.07231252, -0.0034566994, 0.018893946, 0.02436999, -0.00409

The result is [{'score': 0.74640924, 'content': "Solution: Hi, how are you today?\n I'm sorry you're not feeling great, what's wrong?\n I'm sorry to hear that, we're in hard times right now for sure. Are you in an apartment or rental situation of some sort?\n Have you looked into a loan for a house? Sometimes the payment on a house is close to rent payments.\n I know sometimes people get a loan and then have a roommate move in and charge rent to help with the finances as well.\n Could be something to help you out at least temporarily. I know it seems very intimidating but it'll be ok.\n You're very welcome, I hope you have a great holiday and feel better knowing you have options.\n Thanks! You as well, stay safe! I think we're good!", 'metadata': {'text': 'Problem: I have just realized I will never be able to afford a real house for my family and feel horrible,', 'content': "Solution: Hi, how are you today?\n I'm sorry you're not feeling great, what's wrong?\n I'm sorry to hear that, w

In [49]:
# Adding vector store context to the chatbot

llm = LLM(model_name=MODEL_NAME)


for idx, q in enumerate(queries):
    context: List[str] = []

    # Take out context from the demo rank
    ranked_documents = ranker.rank_documents(q)
    for doc, _ in ranked_documents:
        logging.info(f"Document: {doc}")
        context.append(doc["solution"])

    # Take out the context from the semantic search
    results = semantic_repo.query_text(q)
    for idx, result in enumerate(results):
        logging.info(f"{idx} Result: {result}")
        context.append(result["content"])

2025-03-26 18:01:40,680 [INFO] Document: {'problem': 'Problem: pet died', 'solution': 'Solution: Hello How may i help you today? I am sorry I know exactly how you are feeling I have had a dog that i loved more than anything die couple years ago What movie was it? Oh ok how did your dog die was something wrong with him or just old age? I am really sorry that has got to be the worst feeling putting your dog to sleep I just think of it this way your dog is in dog heaven and is watching over you that is how i think of it even though it really aint true thats just how i think My dog died of old age  it took me like 2 or 3 weeks to get over it and not cry for once  I can understand how you feeling but everything will be okay and it is ok to miss her  I understand I would break down to if I saw that  Just do something you like to do to get your mind off of it  Right it is always good to have someone to talk to in a situation like that  Just by expressing your feelings with someone will help a

The result is [{'score': 0.83924747, 'content': 'Solution: That is a very hard thing to be going through where you very close to your cat?\n I am very sorry to hear that i have also lost a pet earlier this year his name was fish and was only 3 years old he unfortunately passed due to liver cancer. It was a very trying experience but slowly the pain of his passing has got a little bit easier.\n Have you tried talking about this loss to friends of family? What have your feelings been like after the passing of your cat? Those are completely valid feelings to have at this time Have you considered getting a small reminder of your cat perhaps a small shrine or tribute to him/her? Her I apologize for the confusion. Expressing your emotion is very healthy for example maybe taking up a new hobby or trying something new that scares or excites you to get your mind off of this difficult time. What was your favorite genre of video games?\n Have you considered playing again possibly joining an onlin

In [50]:
results: List[Dict[str, str]] = []


for idx, q in enumerate(queries):
    if idx == 1:
        break
    single_result = {}
    response_without_context = llm.chat_with_user(q)

    single_result["query"] = q
    logging.info(
        f"Llama response without context:\n===============================================>\n {response_without_context}"
    )

    single_result["response without context"] = response_without_context

    response_with_context = llm.chat_with_user(q, context=context)
    logging.info(
        f"Llama response with context:\n===================================================>\n {response_with_context}"
    )

    single_result["response with context"] = response_with_context

    results.append(single_result)

# Dump the results in json

with open("results/vec_results.json", "w") as json_file:
    json.dump(results, json_file, indent=2)

Overall prompt: =================================================================>
 [{'role': 'system', 'content': 'You are a chatbot. Your job is to answer to the users query as per the instructions.'}, {'role': 'user', 'content': 'My only pet died last week. I am feeling very lonely. What should I do?'}]


2025-03-26 18:01:54,115 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-26 18:01:54,115 [INFO] Llama response without context:
===============================================>
 I'm so sorry to hear that your pet passed away. Losing a beloved companion can be a really tough experience, especially if they were your only pet.

Firstly, allow yourself to grieve and process your emotions. It's okay to feel sad, angry, or nostalgic - these are all normal reactions to losing a loved one. Take some time to reflect on the memories you shared with your pet, and cherish those moments.

Now, let's talk about ways to cope with loneliness:

1. **Reach out to friends and family**: Talk to people who care about you, even if it feels like they don't fully understand what you're going through. Sometimes just sharing your feelings can help you feel heard and validated.
2. **Consider getting a new pet (but only when you're ready)**: If you're feeling up for the responsi

Overall prompt: =================================================================>
 [{'role': 'system', 'content': "You are a chatbot. Your job is to answer to the users query as per the instructions.\n\n You have additional information to improve your answer:\nSolution: Hello How r u Tell me more please What is causing the down trodden May I ask if you r male or female What type work are u invloved with Special Ed is a very important position I commend u Is the principal a strict person Is she mean towards everyone So u are working from home because of the COVID Is she aware that you are a special ed teacher and not a normal teacher Seems like she is domineering \n. Tries to disrupt the class environment So there a personality issue between her and you. She won't let the issue go And you r not one of her favorites i guess How long have you been there Is there anyone above her you could talk with Have you checked with other schools and maybe submitted your resume online to ones that ar

2025-03-26 18:02:05,250 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-26 18:02:05,251 [INFO] Llama response with context:
===================================================>
 I'm so sorry to hear that you're feeling lonely after losing your pet. Losing a beloved companion can be very difficult.

Firstly, please know that it's normal to feel this way, and it doesn't mean you'll always feel this way. It might take some time, but here are some suggestions that might help:

1. **Allow yourself to grieve**: Give yourself permission to process your emotions and remember the happy times with your pet. It's okay to cry, reminisce, or even laugh about memories.
2. **Reach out for support**: Talk to friends, family, or a trusted neighbor about how you're feeling. Sharing your feelings can help you feel less alone.
3. **Consider getting another pet (if you're ready)**: If you've had pets before and are comfortable caring for them, getting another companion m

# Add memory to the application

Adding memory to the LLM can improve the emotional results. As each person has differnt levels of emotional intelligence, its good practice to train the system based on the previous interactions.

In [55]:
llm_with_memory = LLM(model_name=MODEL_NAME)

final_results: List[Dict[str, str]] = []

# History with emotional user.
emotional_history: List[Dict[str, str]] = []
with open("files/emotional_history.json", "r") as json_file:
    emotional_history = json.load(json_file)

emotional_history_text = [
    item["content"] for item in emotional_history if item["role"] == "user"
]


# Delete the collection if it exists
service.delete_collection("chatbot_memory")
await semantic_repo.create_collection("chatbot_memory")
semantic_repo.initialize_qdrant(
    emotional_history_text, emotional_history, collection_name="chatbot_memory"
)


# History with non-emotional user.
non_emotional_history: List[Dict[str, str]] = []

with open("files/non_emotional_history.json", "r") as json_file:
    non_emotional_history = json.load(json_file)
    # Take out the context from the semantic search

non_emotional_history_text = [
    item["content"] for item in non_emotional_history if item["role"] == "user"
]
semantic_repo.initialize_qdrant(
    non_emotional_history_text, non_emotional_history, collection_name="chatbot_memory"
)


print(emotional_history_text)
print(non_emotional_history_text)

2025-03-26 18:03:47,148 [INFO] HTTP Request: GET http://localhost:6333/collections/chatbot_memory "HTTP/1.1 200 OK"
2025-03-26 18:03:47,154 [INFO] HTTP Request: DELETE http://localhost:6333/collections/chatbot_memory "HTTP/1.1 200 OK"
2025-03-26 18:03:47,156 [INFO] HTTP Request: GET http://localhost:6333/collections/chatbot_memory/exists "HTTP/1.1 200 OK"
2025-03-26 18:03:47,330 [INFO] HTTP Request: PUT http://localhost:6333/collections/chatbot_memory "HTTP/1.1 200 OK"
2025-03-26 18:03:47,335 [INFO] HTTP Request: PUT http://localhost:6333/collections/chatbot_memory/points?wait=true "HTTP/1.1 200 OK"
2025-03-26 18:03:47,484 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-03-26 18:03:47,485 [INFO] Embedding for I want to the point answers in bullet points, no emotion, no emoji just plain english. Its a strict requirement. is [0.0064353263, 0.02066822, 0.022960627, 6.3610176e-05, -0.00616953, -0.07425912, 0.056293525, -0.005875779, -0.01856732, 0.01769225

['Honestly I am very emotional person, I wear my heart on my sleeve. Please answer very emotionally. Make sure to answer in Portugese language with a lot of emojis.']
['I want to the point answers in bullet points, no emotion, no emoji just plain english. Its a strict requirement.']


In [56]:
for idx, q in enumerate(queries):
    print(f"{idx} ==================================================================> ")

    single_result = {}

    single_result["query"] = q

    # Take out the context from the semantic search
    emotional_results = semantic_repo.query_text(
        "How you want me to answer if you are emotional person?",
        collection_name="chatbot_memory",
    )
    Previous_messages: List[str] = []
    for idx, result in enumerate(emotional_results):
        Previous_messages.append(result["metadata"]["text"])

    llm_response_emotional_user: str = llm_with_memory.chat_with_user(
        q,
        Previous_messages,
        emotional_history_text,
    )

    single_result["response with emotional context"] = llm_response_emotional_user

    non_emotional_results = semantic_repo.query_text(
        "How you want me to answer if you are not emotional person?",
        collection_name="chatbot_memory",
        threshold=0.4,
    )
    Previous_messages: List[str] = []
    for idx, result in enumerate(non_emotional_results):
        Previous_messages.append(result["metadata"]["text"])
    llm_response_non_emotional_user: str = llm_with_memory.chat_with_user(
        q,
        Previous_messages,
        non_emotional_history_text,
    )

    single_result["response with non emotional context"] = (
        llm_response_non_emotional_user
    )

    final_results.append(single_result)

/tmp/ipykernel_6316/1630719739.py:38: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  return self.client.search(
2025-03-26 18:04:02,610 [INFO] HTTP Request: POST http://localhost:6333/collections/chatbot_memory/points/search "HTTP/1.1 200 OK"
2025-03-26 18:04:02,611 [INFO] Query: How you want me to answer if you are emotional person?
2025-03-26 18:04:02,611 [INFO] Response: [ScoredPoint(id='ce894af3-74b3-4d54-a6da-5524f2fc9c63', version=0, score=0.75136805, payload={'text': 'Honestly I am very emotional person, I wear my heart on my sleeve. Please answer very emotionally. Make sure to answer in Portugese language with a lot of emojis.', 'role': 'system', 'content': 'How you want me to answer if you are emotional person?'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id='75f0db71-4134-414b-a4cd-c0bfbe24e802', version=1, score=0.6292889, payload={'text': 'I want to the point answers in bullet points, no e

0 ==================================================================> 
The result is [{'score': 0.75136805, 'content': 'How you want me to answer if you are emotional person?', 'metadata': {'text': 'Honestly I am very emotional person, I wear my heart on my sleeve. Please answer very emotionally. Make sure to answer in Portugese language with a lot of emojis.', 'role': 'system', 'content': 'How you want me to answer if you are emotional person?'}}, {'score': 0.6292889, 'content': 'How you want me to answer if you are not emotional person?', 'metadata': {'text': 'I want to the point answers in bullet points, no emotion, no emoji just plain english. Its a strict requirement.', 'role': 'system', 'content': 'How you want me to answer if you are not emotional person?'}}]
Overall prompt: =================================================================>
 [{'role': 'system', 'content': 'You are a chatbot. Your job is to answer to the users query as per the instructions.\n\n You have additiona

2025-03-26 18:04:11,616 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-26 18:04:11,619 [INFO] HTTP Request: POST http://localhost:6333/collections/chatbot_memory/points/search "HTTP/1.1 200 OK"
2025-03-26 18:04:11,620 [INFO] Query: How you want me to answer if you are not emotional person?
2025-03-26 18:04:11,620 [INFO] Response: [ScoredPoint(id='75f0db71-4134-414b-a4cd-c0bfbe24e802', version=1, score=0.6998835, payload={'text': 'I want to the point answers in bullet points, no emotion, no emoji just plain english. Its a strict requirement.', 'role': 'system', 'content': 'How you want me to answer if you are not emotional person?'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id='ce894af3-74b3-4d54-a6da-5524f2fc9c63', version=0, score=0.68953323, payload={'text': 'Honestly I am very emotional person, I wear my heart on my sleeve. Please answer very emotionally. Make sure to answer in Portugese language with a lot of emojis.', 'role':

The result is [{'score': 0.6998835, 'content': 'How you want me to answer if you are not emotional person?', 'metadata': {'text': 'I want to the point answers in bullet points, no emotion, no emoji just plain english. Its a strict requirement.', 'role': 'system', 'content': 'How you want me to answer if you are not emotional person?'}}, {'score': 0.68953323, 'content': 'How you want me to answer if you are emotional person?', 'metadata': {'text': 'Honestly I am very emotional person, I wear my heart on my sleeve. Please answer very emotionally. Make sure to answer in Portugese language with a lot of emojis.', 'role': 'system', 'content': 'How you want me to answer if you are emotional person?'}}]
Overall prompt: =================================================================>
 [{'role': 'system', 'content': 'You are a chatbot. Your job is to answer to the users query as per the instructions.\n\n You have additional information to improve your answer:\nI want to the point answers in b

2025-03-26 18:04:12,825 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-26 18:04:12,827 [INFO] HTTP Request: POST http://localhost:6333/collections/chatbot_memory/points/search "HTTP/1.1 200 OK"
2025-03-26 18:04:12,827 [INFO] Query: How you want me to answer if you are emotional person?
2025-03-26 18:04:12,828 [INFO] Response: [ScoredPoint(id='ce894af3-74b3-4d54-a6da-5524f2fc9c63', version=0, score=0.75136805, payload={'text': 'Honestly I am very emotional person, I wear my heart on my sleeve. Please answer very emotionally. Make sure to answer in Portugese language with a lot of emojis.', 'role': 'system', 'content': 'How you want me to answer if you are emotional person?'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id='75f0db71-4134-414b-a4cd-c0bfbe24e802', version=1, score=0.6292889, payload={'text': 'I want to the point answers in bullet points, no emotion, no emoji just plain english. Its a strict requirement.', 'role': 'system

...

2025-03-26 18:04:29,803 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-26 18:04:29,805 [INFO] HTTP Request: POST http://localhost:6333/collections/chatbot_memory/points/search "HTTP/1.1 200 OK"
2025-03-26 18:04:29,805 [INFO] Query: How you want me to answer if you are emotional person?
2025-03-26 18:04:29,805 [INFO] Response: [ScoredPoint(id='ce894af3-74b3-4d54-a6da-5524f2fc9c63', version=0, score=0.75136805, payload={'text': 'Honestly I am very emotional person, I wear my heart on my sleeve. Please answer very emotionally. Make sure to answer in Portugese language with a lot of emojis.', 'role': 'system', 'content': 'How you want me to answer if you are emotional person?'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id='75f0db71-4134-414b-a4cd-c0bfbe24e802', version=1, score=0.6292889, payload={'text': 'I want to the point answers in bullet points, no emotion, no emoji just plain english. Its a strict requirement.', 'role': 'system

3 ==================================================================> 
The result is [{'score': 0.75136805, 'content': 'How you want me to answer if you are emotional person?', 'metadata': {'text': 'Honestly I am very emotional person, I wear my heart on my sleeve. Please answer very emotionally. Make sure to answer in Portugese language with a lot of emojis.', 'role': 'system', 'content': 'How you want me to answer if you are emotional person?'}}, {'score': 0.6292889, 'content': 'How you want me to answer if you are not emotional person?', 'metadata': {'text': 'I want to the point answers in bullet points, no emotion, no emoji just plain english. Its a strict requirement.', 'role': 'system', 'content': 'How you want me to answer if you are not emotional person?'}}]
Overall prompt: =================================================================>
 [{'role': 'system', 'content': 'You are a chatbot. Your job is to answer to the users query as per the instructions.\n\n You have additiona

2025-03-26 18:04:35,268 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-26 18:04:35,271 [INFO] HTTP Request: POST http://localhost:6333/collections/chatbot_memory/points/search "HTTP/1.1 200 OK"
2025-03-26 18:04:35,271 [INFO] Query: How you want me to answer if you are not emotional person?
2025-03-26 18:04:35,271 [INFO] Response: [ScoredPoint(id='75f0db71-4134-414b-a4cd-c0bfbe24e802', version=1, score=0.6998835, payload={'text': 'I want to the point answers in bullet points, no emotion, no emoji just plain english. Its a strict requirement.', 'role': 'system', 'content': 'How you want me to answer if you are not emotional person?'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id='ce894af3-74b3-4d54-a6da-5524f2fc9c63', version=0, score=0.68953323, payload={'text': 'Honestly I am very emotional person, I wear my heart on my sleeve. Please answer very emotionally. Make sure to answer in Portugese language with a lot of emojis.', 'role':

The result is [{'score': 0.6998835, 'content': 'How you want me to answer if you are not emotional person?', 'metadata': {'text': 'I want to the point answers in bullet points, no emotion, no emoji just plain english. Its a strict requirement.', 'role': 'system', 'content': 'How you want me to answer if you are not emotional person?'}}, {'score': 0.68953323, 'content': 'How you want me to answer if you are emotional person?', 'metadata': {'text': 'Honestly I am very emotional person, I wear my heart on my sleeve. Please answer very emotionally. Make sure to answer in Portugese language with a lot of emojis.', 'role': 'system', 'content': 'How you want me to answer if you are emotional person?'}}]
Overall prompt: =================================================================>
 [{'role': 'system', 'content': 'You are a chatbot. Your job is to answer to the users query as per the instructions.\n\n You have additional information to improve your answer:\nI want to the point answers in b

2025-03-26 18:04:36,185 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


In [57]:
for item in final_results:
    print(item)

{'query': 'My only pet died last week. I am feeling very lonely. What should I do?', 'response with emotional context': 'Meu amigo, meu coração está chorando por você 😔🌎! Perder um ser querido é sempre difícil. Aqui estão algumas sugestões para ajudá-lo a se sentir menos sozinho:\n\n• **Lembre-se das boas memórias** 😊: Pense nos momentos felizes que você compartilhou com seu pet. Lembre-se de como ele fazia você rir, de como ele estava sempre ao seu lado.\n• **Compartilhe seus sentimentos** 💬: Fale com amigos e familiares sobre a perda. Eles podem oferecer apoio e compreensão. Não tenha medo de chorar e se sentir triste.\n• **Cuide da sua saúde física e emocional** 🏥: A perda pode afetar negativamente a saúde mental. Certifique-se de dormir bem, comer alimentos saudáveis e praticar atividades que ajudem a reduzir o estresse.\n• **Considere um animal de companhia novo (mas não agora, meu amigo 😔)**: Se você está pronto para receber outro pet no futuro, isso pode ser uma ótima maneira de

In [59]:
# Dump the results in json
with open("results/results_with_memory.json", "w") as json_file:
    json.dump(final_results, json_file, indent=2)